In [7]:
diretorio_datasets = "/home/matteus-paula/Downloads/dados_tcc/remun_docentes"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

> ## Dados Remuneração Docentes Muncipio:<br>
- https://www.fnde.gov.br/index.php/fnde_sistemas/siope/relatorios/arquivos-dados-analiticos

In [8]:
def read_docente(uf):
    df1_remun_docentes = pd.read_csv(f'{diretorio_datasets}/REMUNERACAO_PROFISSIONAIS_EDUCACAO_{uf}_2017.CSV', 
                                     delimiter=";", 
                                     encoding = "ISO-8859-1",
                                     usecols=["CO_MUNICIPIO", 
                                              "NO_PROFISSIONAL", 
                                              "CO_UF", 
                                              "VL_TOTAL", 
                                              "CATEG_PROFISSIONAL", 
                                              "TP_CATEGORIA"])
    list_categ = ['Docente habilitado em curso de pedagogia', 
                  'Docente habilitado em curso de licenciatura plena',
                  'Docente habilitado em curso de nível médio',
                  'Profissionais em efetivo exercício no âmbito da educação infantil e ensino fundamental.']

    if df1_remun_docentes.empty:
         return df1_remun_docentes
    else:
        df2_remun_docentes = df1_remun_docentes[df1_remun_docentes['CATEG_PROFISSIONAL'].isin(list_categ)
                                                 & (df1_remun_docentes['TP_CATEGORIA'] == 'PROFISSIONAIS DO MAGISTÉRIO')]
        df3_remun_docentes = df2_remun_docentes[['CO_MUNICIPIO',
                                                 'NO_PROFISSIONAL', 
                                                 'CO_UF', 
                                                 'VL_TOTAL']]
        df3_remun_docentes['VL_TOTAL'] = df3_remun_docentes['VL_TOTAL'].str.replace(',','.').astype('float')
        df4_remun_docentes = df3_remun_docentes[['CO_MUNICIPIO',
                                                 'NO_PROFISSIONAL', 
                                                 'CO_UF', 
                                                 'VL_TOTAL']] \
                                               .groupby(['CO_MUNICIPIO',
                                                 'NO_PROFISSIONAL', 
                                                 'CO_UF',], as_index=False) \
                                               .agg(lambda x: round(x.mean(), 2))
        return df4_remun_docentes

In [9]:
def media_rem_docente_uf(df):
    if df.empty:
        return df
    else:
        df1 = df[['CO_MUNICIPIO',
                  'CO_UF', 
                  'VL_TOTAL']] \
                .groupby(['CO_MUNICIPIO',
                          'CO_UF',], as_index=False) \
                .agg(lambda x: round(x.quantile(.5), 2))
        return df1


In [10]:
ufs = ['AC',
'AL',
'AP',
'AM',
'BA',
'CE',
'DF',
'ES',
'GO',
'MA',
'MT',
'MS',
'MG',
'PA',
'PB',
'PR',
'PE',
'PI',
'RJ',
'RN',
'RS',
'RO',
'RR',
'SC',
'SP',
'SE',
'TO']

COLUNAS = ['CO_MUNICIPIO',
           'CO_UF', 
           'VL_TOTAL']
df_rem_docentes = pd.DataFrame(columns=COLUNAS)
for uf in ufs:
    df = media_rem_docente_uf(read_docente(uf))
    if df.empty:
        print(uf)
    else:
        df_rem_docentes = df_rem_docentes.append(df)
        print(uf + ' - ' + str(df['CO_UF'].count()))

print(df_rem_docentes['CO_UF'].count())

/home/matteus-paula/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AC - 22
AL - 96
AP - 16
AM - 61
BA - 410
CE - 176
DF
ES - 78
GO - 217
MA - 204
MT - 140
MS - 78
MG - 844
PA - 133
PB - 222
PR - 391
PE - 180
PI - 217
RJ - 91
RN - 158
RS - 494
RO - 52
RR - 15
SC - 290
SP - 639
SE - 74
TO - 135
5433


In [11]:
df_rem_docentes_expurgo = df_rem_docentes[(df_rem_docentes.VL_TOTAL > 10000)]
df_rem_docentes_expurgo['CO_UF'].count()

107

In [12]:
df_rem_docentes_normalizado = df_rem_docentes[(df_rem_docentes.VL_TOTAL <= 10000)]
df_rem_docentes_normalizado['CO_UF'].count()

5326

In [13]:
df_rem_docentes_expurgo.to_csv(f'{diretorio_datasets}/expurgo_remuneracao_media_docentes.csv', sep=';', encoding='utf-8', index=False)
df_rem_docentes_normalizado.to_csv(f'{diretorio_datasets}/remuneracao_media_docentes.csv', sep=';', encoding='utf-8', index=False)